In [25]:
def a_method(a, b)
  a + yield(a, b)
end

a_method(1, 2) {|x, y| (x + y) * 3}

10

只有在调用一个方法时，才可以定义一个块。块会被直接传递给这次方法，该方法可以用`yield`关键字调用这个块。

块可以拥有自己的参数，比如上面例子中的`x`和`y`。当回调块时，你可以像调用方法那样为块提供参数。另外，像方法一样，块的最后一行代码执行的结果会被作为返回值。

在一个方法里，你可以询问当前的方法调用是后包含块。这可以通过`Kernel#block_given?`方法做到

In [2]:
def a_method
  return yield if block_given?
  'no block'
end

p a_method
p a_method {"here's a block!"}

"no block"
"here's a block!"


"here's a block!"

In [3]:
module Kernel
  def with(resource)
    begin
      yield
    ensure
      resource.dispose
    end
  end
end

:with

定义一个块的时候，它会获取环境中的绑定。当块被传给一个方法时，它会带着这些绑定给一块进入方法

In [4]:
def my_method
  x = "Goodbye"
  yield("cruel")
end

x = "hello"
my_method {|y| "#{x}, #{y} wolrd"}

"hello, cruel wolrd"

创建代码块时候，你会获得局部绑定（比如上面的`x`），然后把代码块连同它的绑定传给一个方法。在上面的例子中，代码块的绑定中包括一个名为`x`的变量。虽然在方法中也定义了一个变量`x`，但代码块看到的`x`还是在代码块定义时绑定的`x`，方法中的`x`对于这个代码块来说是不可以见的。

还可以在代码块内定义额外的绑定，但这些绑定在代码块结束时就消失了。

In [5]:
def just_yield
  yield
end

top_level_variable = 1

just_yield do
  top_level_variable += 1
  local_to_block = 1
end

p top_level_variable
local_to_block

2


NameError: undefined local variable or method `local_to_block' for main:Object

基于这样的特性，人们喜欢把代码块称为闭包(`closure`)。换句话说，代码块可以获取局部绑定，并一直带着他们。

下面的例子演示了程序运行时作用域是如何切换的，`Kernel#local_variable`方法用来跟踪绑定的名字：

In [6]:
v1 = 1
class MyClass
  v2 = 2
  p local_variables
  def my_method
    v3 = 3
    p local_variables
  end
  p local_variables
end

obj = MyClass.new
obj.my_method
obj.my_method
local_variables

[:v2]
[:v2]
[:v3]
[:v3]


[:v1, :obj, :top_level_variable, :_i4, :_4, :x, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :___, :_i1, :_1, :__, :_, :_dir_, :_file_, :_ex_, :_pry_, :_out_, :_in_, :_oh, :_ih, :title]

在某些语言(如`Java`和`C#`中)，有“内部作用域(inner scope)”的概念。在内部作用域里可以看到”外部作用域”的变量。但`ruby`没有嵌套式的作用域，它的作用域是截然分开的：一旦进入一个新的作用域，原先的绑定会被被替换为一组新的绑定。这意味着在程序进入`MyClass`之后，`v1`就落在了作用域之外，从而不可见了。

在i定义`MyClass`的作用域中，程序定义了`v2`以及一个方法。因为方法中的代码还没有被执行，所以直到类定义结束前，程序不会再打开一个新的作用域。在方法定义完成之后，用`class`关键字打开的作用域会永远关闭，同时程序回到顶级作用域。

全局变量可以在 **任何作用域**访问

In [7]:
def a_scope
  $var = "some value"
end

def another_scope
  $var
end


:another_scope

In [8]:
a_scope
another_scope

"some value"

谁都可以设置全局变量，你几乎没办法确定是谁修改了他们。因此，能不使用全局变量，就i尽量不要使用。有时可以使用顶级实例变量代替全局变量。它们是顶级对象`main`的实例变量。

In [9]:
@var  = "The top level @var"

def my_method
  @var
end

my_method

"The top level @var"

In [10]:
class MyClass2
  def my_method
    @var = "This is not the top-level @var!"
    $var
    p @var
    p $var
  end
end

myclass = MyClass2.new
myclass.my_method

"This is not the top-level @var!"
"some value"


"some value"

准备地说，程序会在三个地方关闭一个作用域，同事打开一个新的作用域

- 类定义
- 模块定义
- 方法

每当程序进入（或离开）类定义、模块方法、方法时，就会发生作用域切换。这三种情况分别以`class`、`module`和`def`关键字，就会发生作用域切换。这三种情况分别以`class`、`module`和`def`关键字作为标志。每个关键字都对应一个作用域门(`Scope Gate`)。

In [11]:
v1 = 1
class MyClass
  v2 = 2
  local_variables
  def my _method
    v3 = 3
    local_variables
  end
  local_variables
end

obj = MyClass.new
p obj.my_method
local_variables

[:v3]
[:v3]


[:_i9, :_9, :myclass, :_i8, :_8, :_i7, :_7, :_i6, :_6, :_i5, :_5, :v1, :obj, :top_level_variable, :_i4, :_4, :x, :_i3, :_3, :_i2, :_2, :_i, :_ii, :_iii, :___, :_i1, :_1, :__, :_, :_dir_, :_file_, :_ex_, :_pry_, :_out_, :_in_, :_oh, :_ih, :title]

在`class/module`与`def`之间还有一个微妙的区别。在类定义和模块定义中的代码会立即执行。相反，在方法定义的代码不会立即执行。

In [12]:
my_var = "Success"

MyClass = Class.new do
  puts "#{my_var} in the class definition"
  
  define_method :my_method do
    "#{my_var} in the method"
  end
end

MyClass.new.my_method

Success in the class definition


(pry):100: warning: already initialized constant MyClass
(pry):43: warning: previous definition of MyClass was here


"Success in the method"

使用方法调用来替代作用域门，就可以然给一个作用域看到另外一个作用域里的变量。这种技巧称为嵌套文法作用域（`nested lexical scopes`），不过不过很多`Ruby`程序员习惯叫它“扁平化作用域”，表示如果两个作用域挤压在一起，它们就可以共享的变量。也可以简称为**扁平作用域**。

In [13]:
def define_methods
  shared = 0
  Kernel.send :define_method, :counter do
    shared
  end
  
  Kernel.send :define_method, :inc do
    shared += x
  end
end
define_methods

counter
inc(4)
counter

ArgumentError: wrong number of arguments (given 1, expected 0)

这个例子定义了两个**内核方法**，它还使用动态派发法术来访问`Kernel`的私有类方法`defind_method`。`Kernel#counter`和`Kernel#inc`方法都可以看到`shared`变量，但是其他方法却看不见它，因为这个变量被作用域门保护着，这就是使用`define_methods`方法的原因。这种用来共享变量的技巧称为共享作用域。

每个Ruby作用域都包换一组绑定。不同的作用域之间被作用域门（`class`、`module`、`def`）分割开来。

要想让某个绑定穿越作用域，可以使用代码块。一个代码块是一个闭包，当定义一个代码块时，它会捕获当前环境中的绑定,并把这个闭包传递给该方法

可以使用`Class.new`方法代替`class`关键字，用`Module.new`方法代替`module`关键字，用`Module#define_method`方法代替`def`关键字。这就是扁平作用域的，它更是闭包上常用的法术。

如果一个扁平作用域中定义了多个方法，把这些方法用一个作用域门保护起来，它们就可以共享绑定，这种技巧称为共享作用域

In [14]:
class InstanceVal
  def initialize
    @v = 1
  end
end

obj = InstanceVal.new

obj.instance_eval do
  self
  @v
end

1

运行时，代码块的接收者会成为`self`，因此它可以访问接收者的私有方法和实例变量（如`@v`）。因此他可以访问接收者的私有变量（如`@v`）。即使`instance_eval`方法修改了`self`变量，传给`instance_eval`方法的代码块依然可以看到在它定义时的那些绑定，就像其他代码块那样：

In [15]:
v = 2
obj.instance_eval {@v  = v}
obj.instance_eval {@v}

2

上面的三行代码在同一个扁平作用域中执行，因此它们都可以访问局部变量`v`。由于代码块把它的对象作为`self`，所有它们都可以访问`obj`的实例变量`@v`。我们把传递给`instance_eval`方法的代码块称为**上下文探针**，因为它就像是一个深入到对象中的代码片段，并可以对那个对象进行操作。

In [16]:
class C
  def initialize
    @x = 1
  end
end

class D
  def twisted_method
    @y = 2
    C.new.instance_eval {"@x: #{@x}， @y: #{@y}"}
  end
end

D.new.twisted_method

"@x: 1， @y: "

你可能认为`D#twisted_method`中的代码块既可以访问`C`类的实例变量，也可以访问`D`类中的`@y`实例变量，因为他们都在同一个**扁平作用域**里。然而，实例变量是依赖于当前对象`self`的。因此当`instance_eval`方法吧接收者变为当前对象`self`时，调用者的实例变量就落在作用于范围外了。这样，代码块中的`@y`被当成是`C`类的实例变量，此时它并未初始化。因此值还是`nil`。

In [17]:
class D
  def twisted_method
    @y = 2
    C.new.instance_exec(@y) {|y| "@x #{@x}, @y:#{y}"}
  end
end

:twisted_method

In [18]:
D.new.twisted_method

"@x 1, @y:2"

洁净室只是一个用来执行块的环境，其中的方法名或实例变量名有可能和代码块从其环境中带来的名字冲突，它是白板类，几乎没有任何方法。

尽管Ruby中绝大多数都是对象，但是代码块却不是

为了解决这个问题，Ruby在标准库中提供了一个名为`Proc`的类。`Proc`就是由块转化来的对象。你可以把代码块传给`Proc.new`方法来创建一个`Proc`，以后就可以用`Proce#call`方法执行这个由代码块转换而来的对象：

In [19]:
inc = Proc.new {|x| x + 1}
inc.call(2)

3

你还可以用带刺的`lambda`操作符创建`lambda`

In [21]:
dec = lambda {|x| x - 1}
dec.class
dec.call(2)

1

**代码块就像是方法额外的匿名参数**。绝大多数下，在方法中可以通过`yield`语句直接运行一个代码块。但在下面两种情况下，`yield`就力不从心了。

1. 你想把代码块传递给另外一个方法
2. 你想把代码块转换成`Proc`

In [24]:
def math(a, b)
  yield(a, b)
end

def do_math(a, b ,&operation)
  math(a, b, &operation)
end

do_math(2,3) {|x, y|  x * y}

6

如果在调用`do_math`方法时没有附加代码块，那么`&opertaion`参数江北赋值为`nil`，这样`math`方法中的`yield`方法就会失败

如果把这个块转换成`Proc`呢？ 是集山个，如果你在上面的代码引用了`operation`，就已经有了一个`Proc`对象。`&`操作符的意义是：这是一个`Proc对象`,我想把它当成代码块使用。

In [29]:
def my_method(&the_proc)
  the_proc
end

:my_method

In [31]:
p = my_method {|name| "Hello, #{name}!"}
p.class
p.call("Bill")

"Hello, Bill!"

In [35]:
def my_method(greeting)
  "#{greeting}, #{yield}!"
end

my_proc = proc {"Bill"}
my_method("Hello", &my_proc)

"Hello, Bill!"

调用`my_method`方法时，`&`操作符回吧`my_proc`转化为代码块，再把代码块转化给这个方法

In [47]:
require 'highline'

hl = HighLine.new
friends = hl.ask("Friends?", lambda {|s| s.split(',')})
puts "You're friends with: #{friends.inspect}"

Friends?


EOFError: The input stream is exhausted.

`proc`和`lambda`的第二个差别在于它们检查参数的方式。例如，某个`proc`或`lambda`的参数数量为2，这意味者它可以接受两个参数。

In [49]:
p = Proc.new {|a, b| [a, b]}
p.arity

2

如果给这个可调用对象传递三个参数或一个参数，会怎么样？答案很复杂而且有很多特例。如果`lambda`参数数量不对，就会抛出`ArgumentError`错误，而`proc`则会把参数调整称给自己期望的格式

In [1]:
p = Proc.new {|a, b| [a, b]}
p p.call(1,2,3)
p p.call(1)

[1, 2]
[1, nil]


[1, nil]

最后一个可以调用对象家族的成员时方法。如果你不相信方法跟`lambda`一样也是可调用对象，那么就来看看下面这个例子：

```ruby
class MyClass
  def initialize(value)
    @x = value
  end

  def my_method
    @x
  end
end
object = MyClass.new(1)
m = object.method :my_method
m.call
```

通过调用`Kernel#method`方法，可以获得一个用`Method`对象表示的方法。可以以后用`Method#call`对其进行调用。

`Method`对象类似于代码块或者`lambda`。实际上，可以通过`Method#to_proc`方法把`Method`对象转换为`proc`。另外还可以通过`define_method`方法把代码块转换为方法。然而，它们之间有一个重要的区别，`lambda`在定义它的作用域中执行。而`Method`对象会在它自身所在对象的左俞勇中执行。

## 自由方法

自由方法和跟普通方法类似，不过它从最初定义它的类或者模块中脱离了。通过`Method#unbind`方法，可以把一个方法变成自由方法。你可以直接调用`Module#instance_method`方法获得一个自由方法

```ruby
module MyModule
  def my_method
    42
  end
end

unbound = MyModule.instance_method(:my_method)
unbound.class
```

虽然不能**调用**`UnboundMethod`，但他们把它绑定到一个对象上，使之再次成为一个`Method`对象，具体的做法是使用`UnboundMethod#bind`方法把`UnboundMethod`对象绑定到一个对象上。还可以使把`UnboundMethod`对象传给`Method#define_method`方法，从而实现绑定。（使用动态派发在字符串上调用`define_method`），因为它是私有方法。

```ruby
String.send :define_method, :another_method, unbound
"abc".another_method #42
```

有时我们希望包含`Loadable`的类消除自动加载功能。换句话说，不想调用`Loadable#load`方法，而想调用`Kernel#load`方法。`Ruby`没有`uninclude`语句，因此一旦包换了`Loadable`，就无法从祖先链中去掉。不过，`ActiveSupport`用一行代码就解决了问题。

## 可调用对象小结

可调用对象有以下几种：

- 代码块（它们不是真正的对象，但是它是可调用的）在**定义它的作用域中**执行。
- proc，也在定义它的作用域中执行
- lambda：也是`proc`对象，但是和普通的`proc`有细微的区别，它跟块和`proc`一样都是闭包，因此也子啊定义自身的作用域中执行。
- 方法： 绑定与一个对象，在所绑定对象的作用域中执行，他们也可以与这个作用域解除绑定，然后再重新绑定到另一个对点过的作用于上。

不同对象的可调用对象有细微的差别。在方法和`lambda`中，`return`语句从可以调用对象返回。而在块和`proc`中，`return`从定义可调用对象中返回。而在块和`proc`中，`return`从定义可调用对象的原始上下文中返回。另外，不同的可调用对象对于传入参数数量不符有不同的处理方式。其中方法处理的方式最严格，`lambda`相对严格，而`proc`和块要宽容一点。

尽管有这些差别，你还是可以将一种可调用对象转换为另外一种，实现这样功能的方法包括`Proc.new`方法，`Method#to_proc`方法和`&`操作符。




In [16]:
def setup(&block)
  @setups << block
end

def event(description, &block)
  @events << {:descrption => description, :condition => block}
end

@setups = []
@events = []

setup do
  puts "Setting up sky"
  @sky_height = 100
end

setup do
  puts "Setting up mountains"
  @mountains_height = 200
end

event "the sky is falling" do
  @sky_height < 300
end

event "it's getting closer" do
  @sky_height < 0
end

@events.each do |event|
  @setups.each do |setup|
    setup.call
  end
  puts "ALERT: #{event[:description]}" if event[:condition].call
end


Setting up sky
Setting up mountains
ALERT: 
Setting up sky
Setting up mountains


[{:descrption=>"the sky is falling", :condition=>#<Proc:0x13154d0@(pry):298>}, {:descrption=>"it's getting closer", :condition=>#<Proc:0x1315440@(pry):302>}]

为了消除全局变量，你可以使用全局作用域

In [12]:
lambda {
  setups = []
  events = []
  Kernel.send :define_method, :setup do |&block|
    setups  << block
  end
  
  Kernel.send :define_method, :event do |description, &block|
    setups.each do |setup|
      block.call setup
    end
  end
  
  Kernel.send :define_method,:each_setup do |&block|
    setups.each do |setup|
      block.call setup
    end
  end
  
  Kernel.send :define_method,:each_event do |&block|
    events.each do |setup|
      block.call event
    end
  end
  
}.call

each_event do |event|
  each_setup do |setup|
    setup.call
  end
  puts "ALERT: #{event[:description]}" if event[:condition].call
end

[]

在目前的`RedFlag`版本里，时间可以修改其他时间的顶级共享变量
```ruby
event "define a shared variable" do
  @x = 1
end


event "excahnge the variable" do
  @x = @x + 1
end
```
```ruby
each_event do |event|
  env = Object.new
  each_setup do |setup|
    env.instance_eval &setup
  end
  puts "ALERT: #{event{:descrition}}" if env.instance_eval &(event[:condition])
end
```

你希望事件在`setup`中共享变量，但是并不希望时间之间共享不必要的变量。同样，你需要判断这是一项改进还是一个潜在的bug。如果你认为时间之间应该尽可能的保持独立（就像测试用例集合中的每个测试用例那用），那么可能希望在洁净室中执行这些事件。

你还想用给一个`BasicObject`对象取代`Object`对象来充当洁净室。然而，由于`BasicObject`是一个白板类，其中缺少一些常用的方法，比如`puts`方法和其他`Object`类的方法的前提下，才能使用`BasicObject`。